In [ ]:
import heapq
import numpy as np
import networkx as nx
from tqdm import tqdm

import visualize

In [ ]:
class Intersection:
    def __init__(self, name, x, y,):
        self.name = name
        self.x = x
        self.y = y
        
        self.g = np.inf         # Path cost
        self.h = 0              # Heuristic cost
        self.f = 0              # Total cost 
        
        self.parent = None      # Parent node
        self.connections = {}   # Neighbouring stations

    def add_connection(self, intersection, cost):
        self.connections[intersection] = cost

    def __lt__(self, other):
        return self.f < other.f

In [ ]:
def generate_random_intersections(height, width, connection_chance, max_cost=5, seed=0):
    # Create the intersection objects
    np.random.seed(seed)
    intersections = {}
    intersection_names = {}
    name = 0
    for i in range(height):
        for j in range(width):
            intersections[(i, j)] = Intersection(name=name, x=i, y=j)
            intersection_names[name] = (i, j)
            name += 1

    # Add connections between intersections
    for i in range(height):
        for j in range(width):
            has_connection = False
            while not has_connection:
                if i > 0 and np.random.rand() < connection_chance:
                    intersections[(i, j)].add_connection(intersections[(i-1, j)], cost=np.random.randint(1, max_cost))
                    has_connection = True

                if i < height - 1 and np.random.rand() < connection_chance:
                    intersections[(i, j)].add_connection(intersections[(i+1, j)], cost=np.random.randint(1, max_cost))
                    has_connection = True

                if j > 0 and np.random.rand() < connection_chance:
                    intersections[(i, j)].add_connection(intersections[(i, j-1)], cost=np.random.randint(1, max_cost))
                    has_connection = True

                if j < width - 1 and np.random.rand() < connection_chance:
                    intersections[(i, j)].add_connection(intersections[(i, j+1)], cost=np.random.randint(1, max_cost))
                    has_connection = True

    return intersections, intersection_names

In [ ]:
def get_heuristic(a, b):
    """ 
    :param a: node object representing point 1
    :param b: node object representing point 2

    :return: Manhattan distance between two points
    """
    return abs(a.x - b.x) + abs(a.y - b.y)

In [ ]:
def astar(start, end):
    """
    :param start: node object representing start point
    :param end: node object representing end point

    :return: the optimal path found
    """
    open_list = []      # Potential nodes to explore
    closed_list = []    # Nodes that have already been explored

    heapq.heappush(open_list, start)
    start.g = 0         # Starting station has 0 distance to itself

    # Loop until you reach the end
    while open_list:
        # Explore the next frontier station (lowest f value)
        curr = heapq.heappop(open_list)
        closed_list.append(curr)

        # If the end is found, work backwards to build the path
        if curr == end:
            path = []
            while curr:
                path.append(curr.name)
                curr = curr.parent
            return path[::-1]

        # Else, update all neighbours
        for next in curr.connections.keys():
            # Don't explore any stations in the closed list
            if next in closed_list:
                continue

            # Calculate the time to travel to the next station + wait time for switching lines
            # If a shorter path to the next station is found, update its values
            new_distance = curr.g + curr.connections[next]
            if new_distance < next.g:
                next.parent = curr
                next.g = new_distance
                next.h = get_heuristic(next, end)
                next.f = next.g + next.h

                # If the next station isn't in the open list (frontier), add it
                if next not in open_list:
                    heapq.heappush(open_list, next)

    # Path not found
    return None

In [ ]:
# Generate the intersections
intersections, intersection_names = generate_random_intersections(height=10, width=8, connection_chance=0.7, seed=10)
nodes, edges = visualize.get_nodes_and_edges_traffic(intersections)

# Visually compare going from the first intersection to the last intersection
start = 0
end = max(nodes.keys())

# Compute shortest path using A* algorithm
G = visualize.get_graph(nodes, edges, directed=True)
path_a_star = astar(intersections[intersection_names[start]], intersections[intersection_names[end]])
path_cost_astar = sum(G[u][v]['weight'] for u, v in zip(path_a_star[:-1], path_a_star[1:]))
print(f"Shortest path using A*: {path_a_star}, cost: {path_cost_astar}")
visualize.draw_graph_path(G, nodes, path_a_star, directed=True)

# Compute shortest path using Dijkstra's algorithm
G = visualize.get_graph(nodes, edges, directed=True)
path_cost_dijkstra, path_dijkstra = nx.single_source_dijkstra(G, source=start, target=end)
print(f"Shortest path using Dijkstra: {path_dijkstra}, cost: {path_cost_dijkstra}")
visualize.draw_graph_path(G, nodes, path_dijkstra, directed=True)

In [ ]:
# Compare the algorithms over multiple iterations
def compare_algorithms(intersections, intersection_names, nodes, edges, iterations):
    costs_a_star = []
    costs_dijkstra = []
    for i in tqdm(range(iterations)):
        # Regenerate the intersections because the code breaks otherwise
        intersections, intersection_names = generate_random_intersections(height=10, width=8, connection_chance=0.7, seed=i)
        nodes, edges = visualize.get_nodes_and_edges_traffic(intersections)
        G = visualize.get_graph(nodes, edges, directed=True)

        # Choose a random starting and ending point
        start = np.random.choice(list(intersection_names.keys()))
        end = np.random.choice(list(intersection_names.keys()))

        # Calculate the shortest paths and their costs
        path_a_star = astar(intersections[intersection_names[start]], intersections[intersection_names[end]])
        if path_a_star:
            path_cost_astar = sum(G[u][v]['weight'] for u, v in zip(path_a_star[:-1], path_a_star[1:]))
            costs_a_star.append(path_cost_astar)
            
            path_cost_dijkstra, path_dijkstra = nx.single_source_dijkstra(G, source=start, target=end)
            costs_dijkstra.append(path_cost_dijkstra)

    print(f"A* average cost: {np.mean(costs_a_star)}")
    print(f"Dijkstra average cost: {np.mean(costs_dijkstra)}")

compare_algorithms(intersections, intersection_names, nodes, edges, iterations=100)